In [1]:
import time
import polars as pl
import torch
from typing import Optional
from dataclasses import dataclass
from sentence_transformers import SentenceTransformer
from safetensors.torch import load_file
from contextlib import ContextDecorator
import os

class Timer(ContextDecorator):
    """Контекстный менеджер для замера времени выполнения кода."""
    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, *args):
        self.end = time.time()
        print(f"Время выполнения: {self.end - self.start:.6f} секунд")

@dataclass
class SkillsExtractor:
    vacancy_path: str
    embeddings_path: str
    model_path: Optional[str]
    device: str = "cpu"
    vacancies: Optional[pl.DataFrame] = None
    embeddings: Optional[torch.Tensor] = None

    def __post_init__(self):
        self.initialize_model()

    def initialize_model(self):
        """Инициализация модели SentenceTransformer."""
        self.model = SentenceTransformer(
            model_name_or_path=self.model_path,
            device=self.device,
            local_files_only=True,
            trust_remote_code=True
        )

    def load_vacancies_and_embeddings(self):
        """Загрузка вакансий и эмбеддингов с проверкой существования файлов."""
        self.vacancies = pl.read_parquet(self.vacancy_path)
        self.embeddings = load_file(self.embeddings_path)["embeddings"]

    def show_data_info(self):
        """Показать информацию о вакансиях и эмбеддингах."""
        if self.vacancies is not None:
            print(f"Вакансии: {self.vacancies.shape}")
        else:
            print("Вакансии не загружены.")

        if self.embeddings is not None:
            print(f"Эмбеддинги: {self.embeddings.shape}")
        else:
            print("Эмбеддинги не загружены.")

# Пример использования класса с замером времени выполнения
with Timer():
    skills_extractor = SkillsExtractor(
        model_path = "/Users/dl/GitHub/RecSysApp/models/LaBSE-en-ru",
        vacancy_path ="/Users/dl/GitHub/RecSysApp/data/Вакансии.parquet",
        embeddings_path = "/Users/dl/GitHub/RecSysApp/data/Вакансии_эмбеддинги_LaBSE-en-ru.safetensors",
        device = "cpu"
    )
    skills_extractor.load_vacancies_and_embeddings()
    skills_extractor.show_data_info()

/Users/dl/@DmitryRyumin/Python/envs/RecSys/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Вакансии: (164065, 4)
Эмбеддинги: torch.Size([164065, 768])
Время выполнения: 0.650061 секунд


In [2]:
skills_extractor.vacancies

name,id,parent,key_skills
str,i64,str,str
"""Сетевой инженер""",103639272,"""Сетевой инженер""","""Cisco; MikroTik; TCP/IP; Aster…"
"""Сетевой инженер, ЦОД""",104725303,"""Сетевой инженер""","""Linux; Huawei; Администрирован…"
"""Сетевой инженер""",105975500,"""Сетевой инженер""","""Linux; BGP; OSPF; Администриро…"
"""Администратор баз данных""",105833147,"""Сетевой инженер""","""PostgreSQL; Vertica; Greenplum…"
"""Ведущий сетевой инженер""",105792162,"""Сетевой инженер""","""Сетевой инженер; ВОЛС; СКС; Се…"
…,…,…,…
"""Аналитик DWH""",104588058,"""Аналитик""","""SQL; Моделирование бизнес проц…"
"""Аналитик мерчандайзинга""",104881760,"""Аналитик""","""Аналитические исследования; MS…"
"""Системный аналитик/ System ana…",96641028,"""Аналитик""","""SQL; Системный анализ; REST; A…"


In [3]:
skills_extractor.embeddings

tensor([[ 0.0310, -0.7587, -0.1507,  ..., -0.0036,  0.3871,  0.0890],
        [-0.1499, -0.8299, -0.0987,  ...,  0.1147,  0.4014,  0.0745],
        [ 0.0310, -0.7587, -0.1507,  ..., -0.0036,  0.3871,  0.0890],
        ...,
        [-0.6827, -0.8509, -0.6790,  ..., -0.4839,  0.4057,  0.2252],
        [-0.5957, -0.4537, -0.3978,  ...,  0.3135, -0.0458, -0.5043],
        [-0.2930, -0.4706,  0.0170,  ..., -0.0283, -0.7334, -0.4044]])

In [4]:
skills_extractor.embeddings.shape

torch.Size([164065, 768])

In [6]:
skills_extractor.vacancies.select("parent").unique()

parent
str
"""PR-менеджер"""
"""Инженер-конструктор, инженер-п…"
"""Менеджер по компенсациям и льг…"
"""Event-менеджер"""
"""Арт-директор, креативный дире…"
…
"""Программист, разработчик"""
"""Специалист технической поддер…"
"""Руководитель отдела продаж"""


In [7]:
skills_extractor.vacancies

name,id,parent,key_skills
str,i64,str,str
"""Сетевой инженер""",103639272,"""Сетевой инженер""","""Cisco; MikroTik; TCP/IP; Aster…"
"""Сетевой инженер, ЦОД""",104725303,"""Сетевой инженер""","""Linux; Huawei; Администрирован…"
"""Сетевой инженер""",105975500,"""Сетевой инженер""","""Linux; BGP; OSPF; Администриро…"
"""Администратор баз данных""",105833147,"""Сетевой инженер""","""PostgreSQL; Vertica; Greenplum…"
"""Ведущий сетевой инженер""",105792162,"""Сетевой инженер""","""Сетевой инженер; ВОЛС; СКС; Се…"
…,…,…,…
"""Аналитик DWH""",104588058,"""Аналитик""","""SQL; Моделирование бизнес проц…"
"""Аналитик мерчандайзинга""",104881760,"""Аналитик""","""Аналитические исследования; MS…"
"""Системный аналитик/ System ana…",96641028,"""Аналитик""","""SQL; Системный анализ; REST; A…"


In [ ]:
1. пройтись по все parent найти их в name и если нет, то посчитать эмбединги и добавить в конец списка

In [10]:
skills_extractor.vacancies[0]

name,id,parent,key_skills
str,i64,str,str
"""Сетевой инженер""",103639272,"""Сетевой инженер""","""Cisco; MikroTik; TCP/IP; Aster…"


In [17]:
skills_extractor.vacancies[2]

name,id,parent,key_skills
str,i64,str,str
"""Сетевой инженер""",105975500,"""Сетевой инженер""","""Linux; BGP; OSPF; Администриро…"


In [16]:
skills_extractor.embeddings[0] == skills_extractor.embeddings[2]

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, Tr

In [11]:
skills_extractor.vacancies[2]

name,id,parent,key_skills
str,i64,str,str
"""Сетевой инженер""",105975500,"""Сетевой инженер""","""Linux; BGP; OSPF; Администриро…"


In [9]:
skills_extractor.vacancies.group_by("name").head(2).sort("name")

name,id,parent,key_skills
str,i64,str,str
"""""Менеджер по недвижимости""""",100827006,"""Агент по недвижимости""","""Навыки переговоров; Деловая ко…"
"""""Риэлтор/стажер""""",102841396,"""Агент по недвижимости""","""Телефонные переговоры; Клиенто…"
"""""Финансовый менеджер в отделен…",95866733,"""Кредитный специалист""","""Пользователь ПК; Навыки продаж…"
"""""Финансовый менеджер в отделен…",102408858,"""Кредитный специалист""","""Кредитные договоры; Продажа ст…"
"""""Финансовый менеджер в отделен…",101425506,"""Кредитный специалист""","""Активные продажи; Клиентоориен…"
…,…,…,…
"""​​​​​​​​​​​​​​ Программист Бит…",105992588,"""Программист, разработчик""","""Bitrix API; PHP; MySQL; JS"""
"""•Менеджер отдела продаж""",98230715,"""Менеджер по продажам, менеджер…","""Обучение и развитие; Телефонны…"
"""中文翻译""",106008647,"""Переводчик""","""Деловое общение; Деловые комму…"


In [18]:
skills_extractor.embeddings.shape

torch.Size([164065, 768])

In [21]:
import torch
import numpy as np

tensor_np = skills_extractor.embeddings.numpy()

# Находим уникальные строки
unique_vectors = np.unique(tensor_np, axis = 0)

# Количество уникальных векторов
num_unique_vectors = unique_vectors.shape[0]

print(f"Количество уникальных векторов: {num_unique_vectors}")

Количество уникальных векторов: 78970
